In [49]:
import numpy as np
from twitter_scraper import get_tweets
from twitterscraper import query_tweets
import re
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt

In [50]:
# person_of_interest = 'realDonaldTrump'
# tweets_info = query_tweets('from:' + person_of_interest, begindate=dt.date(2018,4,11), enddate=dt.date.today())
# #     # def query_tweets(query, limit=None, begindate=dt.date(2017,1,1), enddate=dt.date.today(), poolsize=20, lang=''):
    
# # tweets = []
# # for tweet_info in tweets_info:
# #     tweet = tweet_info.text
# #     tweets.append(tweet)

# # tweets = np.array(tweets)

# # print(tweets)

In [237]:
tweets_info = get_tweets(person_of_interest, pages = 1)
tweets = []
for tweet_info in list(tweets_info):
    text = tweet_info['text']
#     text = tweet_info.text #Figure out which one to use. It does not work consistently yet
    # Add NEWTWEET to help recognize starting word of tweet for future processing
    tweets.append("NEWTWEET " + text)
    
tweets.append("NEWTWEET") # Ensures that last word in last tweet is tied to end state
    
tweets = np.array(tweets)
num_tweets = len(tweets)
print("Number of tweets: " + str(num_tweets))
print(tweets)

Number of tweets: 21
['NEWTWEET Welcome Prime Minister Abe!pic.twitter.com/538EYyfWDq'
 'NEWTWEET States and Cities throughout our Country are being cheated and treated so badly by online retailers. Very unfair to traditional tax paying stores!'
 'NEWTWEET Great to be with Secretary Mnuchin & former Governor Sununu in Derry, NH on #TaxDay to highlight the tremendous benefits of #TaxCuts for hard working American families.pic.twitter.com/cOxzqYoqqh'
 'NEWTWEET WATCH LIVE: @IvankaTrump and @stevenmnuchin1 host tax event on Tax Day in New Hampshire. (Courtesy: WBZ) https://twitter.com/i/broadcasts/1OdKrZZwgBYKX\xa0…'
 'NEWTWEET ....Congress – House and Senate must quickly pass a legislative fix to ensure violent criminal aliens can be removed from our society. Keep America Safe!'
 'NEWTWEET Today’s Court decision means that Congress must close loopholes that block the removal of dangerous criminal aliens, including aggravated felons. This is a public safety crisis that can only be fixed b

In [239]:
words = []
for tweet in tweets:
    words.extend(tweet.split(" "))
    
#edge case of URL attatched to end of sentence without space: ... American families.img.pic.twitter...
cleaned_words = []
for word in words:
    if ".com" not in word and "www." not in word and "http" not in word and word != '':
        cleaned_words.append(word)
        
words = cleaned_words
cleaned_words_no_repeat = [words[0]] # could be repeated 'NEWTWEET' due to deleting URL's
        
for i in range(1, len(words)):
    if not (words[i] == "NEWTWEET" and words[i] == words[i-1]):
        cleaned_words_no_repeat.append(words[i])
        
words = cleaned_words_no_repeat
words = np.array(words)
# print(words)

In [240]:
G = nx.DiGraph()
# Initialize states with start and end states (assuming no word is 'S' or 'E')
states = ['S', 'E']
for word in words:
    if word not in states and word != "" and word != "NEWTWEET":
        states.append(word)
G.add_nodes_from(states)
# nx.draw(G, with_labels = True)
# print(states)

In [241]:
freq = {}
edges = []

# Edge case for first word in tweet not being connected to Start state

for i in range(len(words) - 1):
    if words[i] == "NEWTWEET":
        edges.append(('S', words[i + 1], 1))
        continue
    elif words[i+1] == "NEWTWEET":
        edges.append((words[i], 'E', 1))
        continue
    elif (words[i], words[i+1]) in freq:
        freq[(words[i], words[i+1])] += 1
    else:
        freq[(words[i], words[i+1])] = 1
        
    if words[i].endswith(('.', '!', '?')):
        edges.append((words[i], 'E', 1))
        edges.append(('S', words[i + 1], 1))
        
edges.append(('E', 'S', 1))

for edge in freq:
    edges.append((edge[0], edge[1], freq[edge]))
    
G.add_weighted_edges_from(edges)
# nx.draw(G, with_labels = True)
# nx.draw(G)

In [242]:
# print(list(G.successors('S')))
# print(list(G.predecessors('E')))

# print(list(G.predecessors('S')))
# print(list(G.successors('E')))

In [243]:
# Creates numpy adjacency matrix of states, where Starting node is row 0 and Ending node is row 1
matrix = nx.to_numpy_matrix(G, nodelist = states)
matrix = np.array(matrix).tolist()
# print(matrix)
# print(np.array(matrix))

In [244]:
# Sanity check to see if adjacency matrix reflects actual state of MC
num = 0
for i in range(len(matrix[0])):
    num += matrix[1][i]
print(num == len(list(G.successors('E')))) 

True


In [245]:
for i in range(len(matrix)):
    if np.array(matrix[i]).sum(axis=0) != 0:
        continue
    else:
        print(i, np.array(matrix[i]).sum(axis=0))
        print('broken state: ' + states[i])
        print(list(G.predecessors(states[i])))

matrix = np.delete(matrix, len(matrix)-1, axis=1)
matrix = np.delete(matrix, len(matrix)-1, axis=0)

In [246]:
matrix = np.array(matrix)

transition_mat = np.array(matrix)/np.array(matrix).sum(axis=1)[:,None]

def get_next_state(i, transition_matrix):
    return np.random.choice(transition_matrix.shape[1], 1, p=transition_matrix[i])[0]

def perform_random_walk(s, e, transition_matrix):
    curr_state = s
    l = []
    while curr_state != e:
        curr_state = get_next_state(curr_state, transition_matrix)
        if curr_state != s:
            l.append(curr_state)
    return np.array(l)

def generate_sentence():
    return ''.join([states[i] + ' ' for i in perform_random_walk(0, 1, transition_mat) if i != 1])

In [247]:
for i in range(0, 5):
    tweet = generate_sentence()
    if tweet.endswith('..'):
        tweet += ' ' + generate(sentence)
    print(tweet)

(Courtesy: WBZ) 
#MAGA 
Crumbs are being cheated and confusion of the old, broken code. Check out at 51% Approval despite the Border. 
Syrian President Bashar al-Assad’s April 7 attack on #TaxDay to file using the new: tax law, the last year you will generate greater economic opportunities for American 
Employment is the Border. The high crime rate will only get higher. Much wanted Wall in with Prime Minister Abe of Japan, a truly fine gentleman! 
